# 1 start pods on azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr8='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr8='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## add auth tokens for the controller of uoa-poc2

### create new `secrets/auth-tokens.json`

example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "f8ndLS3Dgvmy434lRJhGqbsBDuy6oOXX",
          "allowed_paths": [
            "^/orion/.*$",
            "^/idas/.*$",
            "^/comet/.*$"
          ]
        },
        {
          "token": "TISstc1030",
          "allowed_paths": [
            "^/controller/.*$"
          ]
        },
        {
          "token": "lAra2vvfFCoNYUBZDRSVg0kr0UYXQJeZ",
          "allowed_paths": [
            "^/controller/.*$"
          ]
        }
      ],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "kibana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "kjOocePR",
          "password": "4wm4wDPFiIgKuFwM",
          "allowed_paths": [
            "^.*$"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "grafana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": [
          "^.*$"
        ]
      }
    }
  },
  {
    "host": "zaico\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "dA9edLTb",
          "password": "pnFj8HB1y8Q9Ch8r",
          "allowed_paths": [
            "^.*$"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": [
          "^.*/static/.*$"
        ]
      }
    }
  },
  {
    "host": "robotui\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "vB4xNAlx",
          "password": "jOvGeNTFoFNBzF0P",
          "allowed_paths": [
            "^.*$"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": [
          "^.*/static/.*$"
        ]
      }
    }
  }
]
```

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens|=.+[
  {
    "token": "'$(randomstr32)'",
    "allowed_paths": ["^/controller/.*$"]
  }
]' >> /tmp/auth-tokens.json.1
cat /tmp/auth-tokens.json.1 | jq '.|=.+[{
  "host": "zaico\\..+$",
  "settings": {
    "bearer_tokens": [],
    "basic_auths": [
      {
        "username": "'$(randomstr8)'",
        "password": "'$(randomstr16)'",
        "allowed_paths": ["^.*$"]
      }
    ],
    "no_auths": {
      "allowed_paths": ["^.*/static/.*$"]
    }
  }
},
{
  "host": "robotui\\..+$",
  "settings": {
    "bearer_tokens": [],
    "basic_auths": [
      {
        "username": "'$(randomstr8)'",
        "password": "'$(randomstr16)'",
        "allowed_paths": ["^.*$"]
      }
    ],
    "no_auths": {
      "allowed_paths": ["^.*/static/.*$"]
    }
  }
}]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json
rm /tmp/auth-tokens.json.1

## change the auth-tokens to kubernetes secrets

### watch `auth` log

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "kubectl logs -f -lapp=auth --all-containers=true"

### delete and re-register auth-tokens to kubernetes secrets

In [ ]:
kubectl delete secret auth-tokens

In [ ]:
kubectl create secret generic auth-tokens --from-file=${CORE_ROOT}/secrets/auth-tokens.json

### confirm the token will be reloaded

**wait a few minutes until the change of secret is detected by Kubernetes.**  
When the new secret is detected, the tokens of auth will be reloaded automatically.

log messages like below will be shown after tokens is reloaded.

```
...
--------
2019/08/06 02:28:00 hosts: [.*]
--------
2019/08/06 02:28:00 bearerTokenAllowedPaths: map[.*:map[OnjhCAf8oIRpwBfbFDCh7hu5kh3e9TFM:[^/orion/.*$ ^/idas/.*$ ^/comet/.*$] BcC7SFz7tWAqNtsmQOqbfDCgXqAUyVyn:[^/controller/.*$]]]
--------
2019/08/06 02:28:00 basicAuthPaths, map[]
--------
2019/08/06 02:28:00 noAuthPaths, map[.*:[]]
--------
```

## start robot-controller on azure AKS

In [ ]:
kubectl apply -f controller/robot-controller-azure-service.yaml

In [ ]:
envsubst < controller/robot-controller-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=robot-controller

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
robot-controller-5c7fd7df75-59f7s   1/1     Running   0          26s
robot-controller-5c7fd7df75-lwjs6   1/1     Running   0          26s
robot-controller-5c7fd7df75-wlxfq   1/1     Running   0          26s
```

In [ ]:
kubectl get services -l app=robot-controller

example)
```
NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
robot-controller   ClusterIP   10.101.151.122   <none>        3000/TCP   3m30s
```

## start zaico-extension on azure AKS

notice) replace XXXXXXXXXXXXXXXXXXXX with the 'REST API token' of `zaico.co.jp`

In [ ]:
export ZAICO_TOKEN="XXXXXXXXXXXXXXXXXXXX"

In [ ]:
export SHIPMENTAPI_TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)

In [ ]:
export DESTINATIONS='[{"id": 0, "name": ""}, {"id": 1, "name": "会議室1"}, {"id": 2, "name": "会議室2"}, {"id": 3, "name": "会議室3"}]'

In [ ]:
kubectl apply -f zaico-extensions/zaico-extensions-azure-service.yaml

In [ ]:
envsubst < zaico-extensions/zaico-extensions-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=zaico-extensions

example)
```
NAME                               READY   STATUS    RESTARTS   AGE
zaico-extensions-9b9964947-598q8   1/1     Running   0          15m
zaico-extensions-9b9964947-89gqs   1/1     Running   0          15m
zaico-extensions-9b9964947-dcj75   1/1     Running   0          15m
```

In [ ]:
kubectl get services -l app=zaico-extensions

example)
```
NAME               TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
zaico-extensions   ClusterIP   10.0.103.45   <none>        3000/TCP   6m43s
```

### register DNS A Record for zaico-extensions

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "zaico" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup zaico.${DOMAIN}

In [ ]:
curl -i https://zaico.${DOMAIN}

example)
```
HTTP/1.1 401 Unauthorized
www-authenticate: Basic realm="basic authentication required"
content-length: 0
date: Thu, 10 Oct 2019 08:07:59 GMT
server: envoy
```

## start robotiui on azure AKS

In [ ]:
kubectl apply -f robotui/robotui-azure-service.yaml

In [ ]:
envsubst < robotui/robotui-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=robotui

example)
```
NAME                      READY   STATUS    RESTARTS   AGE
robotui-78b7ff5d5-7rm2c   1/1     Running   0          18s
robotui-78b7ff5d5-92l88   1/1     Running   0          18s
robotui-78b7ff5d5-zwj9d   1/1     Running   0          18s
```

In [ ]:
kubectl get services -l app=robotui

example)
```
NAME      TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
robotui   ClusterIP   10.0.252.198   <none>        8080/TCP   40d
```

### register DNS A Record for zaico-extensions

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "robotui" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup robotui.${DOMAIN}

In [ ]:
curl -i https://robotui.${DOMAIN}

example)
```
HTTP/1.1 401 Unauthorized
www-authenticate: Basic realm="basic authentication required"
content-length: 0
date: Thu, 10 Oct 2019 08:07:59 GMT
server: envoy
```